In [1]:
suppressMessages(library(rwwa))
png_res <- 240

# load covariates
gmst <- read.table("ts-obs/gmst.dat", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2026]
nao <- load_ts("ts-obs/med-storms_nao-djf_era5-stn.dat", col.names = c("year", "nao"))

cov_df <- merge(gmst, nao)
cov_df$gmst <- cov_df$gmst - cov_df$gmst[cov_df$year == 2026]
cov_df$nao_res <- resid(lm(nao ~ gmst, cov_df))

# define factual & counterfactual climates
cov_2026 <- cov_df[cov_df$year == 2026,c(-1),drop = F]
cov_cf <- rbind("pi" = cov_2026 - c(1.3,0,0),
                "naoneutral" = c(cov_2026$gmst, 0,0),
                "pineutral" = c(-1.3, 0,0))

# Tables of results

In [2]:
varnm <- "rx1day-ondjfm"
rnm <- "s"

res <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")
res <- res[!grepl("naodet", rownames(res)),]
res <- res[grepl("era5|eobs", rownames(res)),]

In [5]:
cols <- c("event", "return.period", "dI.rel.pi", "dI.rel.naoneutral", "PR.pi", "PR.naoneutral")
tbl <- data.frame(sapply(cols, function(cnm) {
    apply(signif(res[,grepl(cnm, colnames(res)),drop = F], 3), 1, function(r) {
        paste0(r[1], " (",r[2], ", ", r[3], ")")
    })
}))
tbl$event <- gsub(" .+","",tbl$event)
tbl[c(5,2,4,1,3),]

,event,return.period,dI.rel.pi,dI.rel.naoneutral,PR.pi,PR.naoneutral
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
era5_gmst+nao,52,"3.22 (1.56, 9.35)","19.1 (-1.01, 47.3)","1.61 (-1.75, 5.29)","3.45 (0.925, Inf)","1.09 (0.888, 1.36)"
eobs_gmst+nao,45.4,"1.96 (1.29, 4.05)","39.3 (16.4, 70.5)","2.66 (-1.04, 6.67)","7.12 (2.71, 46.9)","1.1 (0.958, 1.35)"
era5_gmst-only,52,"3.58 (1.81, 10.4)","17 (-1.68, 40.5)","0 (0, 0)","3.18 (0.885, 407)","1 (1, 1)"
eobs_gmst-only,45.4,"2.21 (1.43, 4.78)","35.4 (12.4, 61.6)","0 (0, 0)","6.56 (2.24, 39.6)","1 (1, 1)"
eobs-1920_gmst-only,45.4,"1.93 (1.35, 3.48)","43.3 (22.1, 68.1)","0 (0, 0)","8.86 (3.88, 24)","1 (1, 1)"


# Figures for obs discussion

## Trends vs covariates

In [6]:
varnm <- "rx1day-ondjfm"

for (ds_nm in c("eobs", "era5")[1]) {
    for (rnm in c("s","n")[1]) {
    
        df <- merge(cov_df, load_ts(paste0("ts-obs/med-storms_",varnm,"_",rnm,"_",ds_nm,".dat"), col.names = c("year", "x")))
        mdl <- fit_ns("gev", "fixeddisp", data = df, covnm = c("gmst", "nao"), varnm = "x", lower = F)
        
        ylim <- list("s" = c(0,60), "n" = c(0,100))[[rnm]]

        if (ds_nm == "eobs" & rnm == "s") ev <- NA else ev <- mdl$ev
        
        png(paste0("fig-obs/sr-trendplots_",varnm,"_",rnm,"_",ds_nm,".png"), h = png_res, w = png_res * 3); {
            prep_window(c(1,3), cex.axis = 1.25, mar = c(4,2,3,1), oma = c(0,3,0,0))

            plot_covtrend(mdl, "gmst", ylab = "", xlab = "", legend_pos = NA, ylim = ylim, ev = ev)
            mtext("Maximum 1-day precipitation (mm)", side = 2, line = 3.25)
            mtext("GMST anomaly wrt 2026", side = 1, line = 2.75)
            mtext("(a)", side = 3, line = 0.75, adj = 0, font = 2)
            
            plot_covtrend(mdl, "nao", ylab = "", xlab = "", legend_pos = NA, ylim = ylim, ev = ev)
            mtext("Normalised NAO index", side = 1, line = 2.75)
            mtext("(b)", side = 3, line = 0.75, adj = 0, font = 2)
            
            plot_trend(mdl, ylab = "", xlab = "", legend_pos = NA, ylim = ylim, ev = ev, ev_year = 2026)
            mtext("Year", side = 1, line = 2.75)
            mtext("(c)", side = 3, line = 0.75, adj = 0, font = 2)
            
        }; dev.off()
    }
}

## Return level plots

In [7]:
for (ds_nm in c("eobs", "era5")) {
    for (rnm in c("s","n")) {
    
        df <- merge(cov_df, load_ts(paste0("ts-obs/med-storms_",varnm,"_",rnm,"_",ds_nm,".dat"), col.names = c("year", "x")))
        mdl <- fit_ns("gev", "fixeddisp", data = df, covnm = c("gmst", "nao"), varnm = "x", lower = F)
        
        ylim <- list("s" = c(0,60), "n" = c(0,100))[[rnm]]
        
        png(paste0("fig-obs/sr-rlplots_",varnm,"_",rnm,"_", ds_nm,".png"), h = png_res, w = png_res * 3); {
            prep_window(c(1,2), cex.axis = 1.25, mar = c(4,2,3,1), oma = c(0,3,0,0))
        
            plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["pi",,drop = F], ylab = "", legend_pos = NA, nsamp = 500)
            mtext("Maximum 1-day precipitation (mm)", side = 2, line = 3.25)
            mtext("(a)", side = 3, line = 0.75, adj = 0, font = 2)
        
            plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["naoneutral",,drop = F], ylab = "", legend_pos = NA, nsamp = 500)
            mtext("(b)", side = 3, line = 0.75, adj = 0, font = 2)
            
        }; dev.off()
    }
}

# Model evaluation tables

In [139]:
varnm <- "pr-ndj"
rnm <- "patagonia"

obs_res <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")[c("gmst-only"),,drop = F]
mdl_res <- read.csv(paste0("res-cordex_",varnm,"_gmst-only_",rnm,"_with-eval.csv"), row.names = "X.1")

In [140]:
obs_df <- data.frame("Event" = signif(obs_res$event.magnitude_est,3),
                     "Seasonal" = "",
                     "Spatial" = "", 
                     "Scale parameter" = apply(signif(obs_res[,grepl("sigma0", colnames(obs_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Shape parameter" = apply(signif(obs_res[,grepl("shape", colnames(obs_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Overall" = "",
                     "Keep" = "")
rownames(obs_df) <- c("ERA5-Land")

mdl_df <- data.frame("Event" = signif(mdl_res$rp_value,3),
                     "Seasonal" = mdl_res$seasonal,
                     "Spatial" = mdl_res$spatial, 
                     "Scale parameter" = apply(signif(mdl_res[,grepl("sigma0", colnames(mdl_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Shape parameter" = apply(signif(mdl_res[,grepl("shape", colnames(mdl_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Overall" = mdl_res$cat,
                     "Keep" = mdl_res$include)
rownames(mdl_df) <- gsub("i1p1_", "i1p1 ",rownames(mdl_df))

spacer_df <- data.frame(sapply(colnames(obs_df), function(cnm) rep("",2)), row.names = c("", "CORDEX"))

# Model results

In [258]:
varnm <- "pr-ndj"
rnm <- "chile2026"

obs_res <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")[c("gmst-only"),,drop = F]

mdl_res <- read.csv(paste0("res-cordex_",varnm,"_gmst-only_",rnm,"_with-eval.csv"), row.names = "X.1")
print(nrow(mdl_res))
mdl_res <- mdl_res[mdl_res$include == "Y",]
print(nrow(mdl_res))

[1] 18
[1] 8


In [250]:
cols <- switch(varnm, "hdwi" = c("dI.abs", "PR"), "pr-ndj" = c("dI.rel", "PR"))

mdl_df <- data.frame(cbind(sapply(cols, function(cnm) apply(signif(mdl_res[,grepl(paste0("attr_",cnm), colnames(mdl_res)),drop = F], 3), 1, 
                                                           function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")))),
                     cbind(sapply(cols, function(cnm) apply(signif(mdl_res[,grepl(paste0("proj_",cnm), colnames(mdl_res)),drop = F], 3), 1, 
                                                           function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")))))
rownames(mdl_df) <- gsub("i1p1_", "i1p1 ",rownames(mdl_df))


In [244]:
obs_df <- data.frame(t(sapply(cols, function(cnm) apply(signif(obs_res[,grepl(cnm, colnames(obs_res)),drop = F], 3), 1, 
                                                        function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")))))
colnames(obs_df) <- colnames(mdl_df)[1:length(colnames(obs_df))]
rownames(obs_df) <- c("ERA5-Land")


In [245]:
df <- data.frame(rbind.fill(obs_df, mdl_df), row.names = c(rownames(obs_df), rownames(mdl_df)))

In [246]:
cat(rnm,varnm)
df[,c(1,3,2,4)]

patagonia pr-ndj

,dI.rel,dI.rel.1,PR,PR.1
,<chr>,<chr>,<chr>,<chr>
ERA5-Land,"-35 (-58.5, -3.46)",NA,"9.08 (1.22, 104)",NA
SAM-22_HadGEM2-ES_r1i1p1 RegCM4-7,"-15.3 (-31.1, 3.01)","-8.8 (-16.2, -1.46)","2.6 (0.863, 17.3)","1.45 (1.07, 1.93)"
SAM-22_HadGEM2-ES_r1i1p1 REMO2015,"-14.6 (-30, 3.18)","-11.5 (-19, -4.25)","2.49 (0.853, 14.6)","1.64 (1.22, 2.17)"
SAM-22_NorESM1-M_r1i1p1 RegCM4-7,"-6.36 (-27.3, 17.6)","-15.4 (-26, -5.09)","1.53 (0.44, 18.1)","1.92 (1.29, 2.68)"
SAM-22_NorESM1-M_r1i1p1 REMO2015,"-8.62 (-31.3, 16.6)","-16 (-27, -5.73)","1.79 (0.449, 20.4)","1.96 (1.34, 2.75)"
SAM-44_HadGEM2-ES_r1i1p1 RegCM4-3,"-8.72 (-28, 14.6)","-3.47 (-11.6, 3.44)","1.56 (0.576, 8.32)","1.17 (0.841, 1.64)"
